In [50]:
import pandas as pd
import numpy as np
import sklearn
import xgboost as xgb

In [1]:
# The initial dataset
df = pd.read_csv("final_85.csv", on_bad_lines='skip')

NameError: name 'pd' is not defined

In [38]:
df

,REVEL_score,CADD_RAW,EVE_SCORE,PolyPhen,SIFT,CLIN_SIG
0,0.301301,0.476263,0.292636,0.009,0.14,likely_benign
1,0.301301,0.476263,0.292636,0.027,0.04,likely_benign
2,0.301301,0.476263,0.292636,0.015,0.13,likely_benign
3,0.301301,0.476263,0.292636,0.013,0.10,likely_benign
4,0.301301,0.476263,0.292636,0.009,0.04,likely_benign
...,...,...,...,...,...,...
490,1.372880,14.900000,6.598533,0.000,0.00,pathogenic
491,1.372880,14.900000,6.598533,0.000,0.00,pathogenic
492,1.372880,14.900000,6.598533,0.000,0.00,pathogenic
493,1.372880,14.900000,6.598533,0.000,0.00,pathogenic


In [12]:
# The target column is Clin_SIG, others are some mutagenicity scores that we expect to make impact
df = df[['REVEL_score', 'CADD_RAW', 'EVE_SCORE', 'PolyPhen', 'SIFT', 'CLIN_SIG']]

In [13]:
# check how many NaN in each column
for column in df.columns:
    nan_count = df[column].isna().sum()
    print(f"Column {column} has {nan_count} NaN values.")

Column REVEL_score has 0 NaN values.
Column CADD_RAW has 86 NaN values.
Column EVE_SCORE has 79 NaN values.
Column PolyPhen has 2 NaN values.
Column SIFT has 0 NaN values.
Column CLIN_SIG has 0 NaN values.


In [37]:
df.shape

(460, 6)

In [ ]:
# As we see most of NaN in the target feature. Need to decide wheather should delete rows with NaN or substitute

In [4]:
# check all unique Clin_Sig values
df['CLIN_SIG'].unique()

array(['likely_benign',
       'conflicting_interpretations_of_pathogenicity&uncertain_significance',
       'likely_pathogenic',
       'likely_benign&uncertain_significance&conflicting_interpretations_of_pathogenicity',
       'conflicting_interpretations_of_pathogenicity&uncertain_significance&benign',
       'pathogenic', 'benign/likely_benign&benign',
       'benign/likely_benign', 'benign', 'not_provided&likely_pathogenic',
       'pathogenic/likely_pathogenic&likely_pathogenic&pathogenic',
       'likely_benign&benign/likely_benign',
       'benign&conflicting_interpretations_of_pathogenicity&benign/likely_benign',
       'likely_pathogenic&conflicting_interpretations_of_pathogenicity',
       'likely_benign&conflicting_interpretations_of_pathogenicity',
       'benign&likely_benign',
       'conflicting_interpretations_of_pathogenicity',
       'pathogenic&likely_pathogenic',
       'likely_pathogenic&uncertain_significance'], dtype=object)

In [16]:
(df['CLIN_SIG'] == 'not_provided').sum() 

126

In [17]:
# drop all rows with not_provided in CLIN_SIG
df = df[~(df == 'not_provided').any(axis=1)]

In [17]:
# # Use Frequency encoding
# freq = df['CLIN_SIG'].value_counts(normalize=True)
# df['CLIN_SIG'] = df['CLIN_SIG'].map(freq)

In [ ]:
# Now we need to substitute NaN in other columns

In [26]:
# check all unique Revel values
df['REVEL_score'].unique()

array([0.301301  , 0.591591  , 0.928928  , 0.159159  , 0.086086  ,
       0.307307  , 0.35      , 0.025     , 0.019     , 0.119     ,
       0.279279  , 0.463     , 0.061061  , 0.543543  , 0.205     ,
       0.071     , 0.131131  , 0.91      , 0.913     , 0.784     ,
       0.223223  , 0.36595966, 0.226019  , 0.52685759, 0.30085605,
       0.88892072, 0.09947075, 0.97068845, 0.26240859, 0.63626839,
       0.933     , 0.622     , 0.037     , 0.60560561, 0.096     ,
       0.2002    , 0.064064  , 0.686     , 0.103     , 0.04704705,
       0.929     , 0.87187187, 0.094     , 0.05605606, 0.072     ,
       0.8559543 , 0.569     , 0.31      , 1.422     , 0.342     ,
       0.032     , 1.514     , 0.598     , 0.309     , 0.181     ,
       0.916     , 0.393     , 0.129     , 0.5045045 , 0.601601  ,
       0.113113  , 0.067067  , 0.593593  , 0.052052  , 4.211689  ,
       4.351954  , 1.37288   ])

In [19]:
df['REVEL_score'] = df['REVEL_score'].apply(lambda x: x.replace('&0.', ''))

In [20]:
df['REVEL_score'] = df['REVEL_score'].apply(lambda x: x.replace('&.', ''))

In [21]:
df['REVEL_score'] = df['REVEL_score'].apply(lambda x: x.replace('.&', '0'))

In [25]:
df['REVEL_score'] = df['REVEL_score'].apply(lambda x: float(x))

In [31]:
df['CADD_RAW'].unique()

array([ 0.476263  ,  2.234953  ,  4.302908  , -0.578957  , -0.699148  ,
        3.0311685 ,  0.45267433,  0.46653971, -1.115572  ,  2.03105   ,
        3.760569  ,  3.521154  ,  2.201807  ,  4.732251  ,  1.738637  ,
        2.020851  , -1.26516   ,  3.618528  ,  3.542814  ,  2.539535  ,
        1.673065  ,  0.83231732,  0.20367536,  0.3970337 ,  0.28489046,
        0.19817939,  0.50936578,  0.07624982,  0.51145022,  0.66716996,
        0.13055818,  0.09241236,  3.183496  ,  3.783392  ,  1.613246  ,
        3.924563  ,  1.617616  ,  3.713111  ,  4.111003  ,  2.413458  ,
        3.661225  ,  3.244816  ,  0.843524  ,  2.871921  ,  1.844133  ,
        0.8267593 ,  0.76492492,  0.18634018,  0.19708022, 28.8       ,
       30.        , 14.9       ])

In [57]:
# drop all str values
df = df[df['CADD_RAW'] != 'Uncertain']

In [28]:
# convert every value to numeric
df['CADD_RAW'] = pd.to_numeric(df['CADD_RAW'], errors='coerce')

In [30]:
# substitute nan with mean
df['CADD_RAW'] = df['CADD_RAW'].fillna(df['CADD_RAW'].mean())

In [ ]:
# substitute nan by mean

In [40]:
df['EVE_SCORE'].unique()

array([ 0.2926359 ,  0.36980795,  0.90950153,  6.59853329,  4.636202  ,
        1.882832  ,  1.269156  ,  0.55985002,  0.20856665,  0.93136394,
        0.91849847,  0.92019832,  0.10468851,  3.590577  ,  2.283556  ,
        4.248416  ,  4.656308  ,  5.591318  ,  1.153018  ,  4.806123  ,
        3.375517  ,  4.153694  , 28.7       , 26.3       ,  1.989     ,
       24.3       , 23.6       , 17.64      , 12.49      , 25.1       ,
       16.01      ,  3.559     , 29.5       , 32.        , 11.68      ,
        2.168139  ,  0.973211  , 29.4       ,  0.20054717,  0.98522396,
        0.83501945,  0.11569733,  0.58218742,  0.69434573,  4.41506   ,
        4.305868  ,  1.67579   ,  0.724283  ,  3.740603  ,  1.516408  ])

In [41]:
# convert every value to numeric
df['EVE_SCORE'] = pd.to_numeric(df['EVE_SCORE'], errors='coerce')

In [36]:
# substitute nan with mean
df['EVE_SCORE'] = df['EVE_SCORE'].fillna(df['EVE_SCORE'].mean())

In [38]:
# convert every value to numeric
df['EVE_SCORE'] = pd.to_numeric(df['EVE_SCORE'], errors='coerce')

In [47]:
df['PolyPhen'].unique()

array([0.009    , 0.027    , 0.015    , 0.013    , 0.007    , 0.012    ,
       0.025    , 0.997    , 0.067    , 0.577    , 0.763    , 1.       ,
       0.998    , 0.037    , 0.017    , 0.       , 0.001    , 0.031    ,
       0.583    , 0.003    , 0.095    , 0.155    , 0.204    , 0.258    ,
       0.712    , 0.412    , 0.08     , 0.342    , 0.341    , 0.074    ,
       0.981    , 0.999    , 0.841    , 0.057    , 0.4968499, 0.809    ,
       0.995    , 0.9      , 0.991    , 0.978    , 0.996    , 0.994    ,
       0.958    , 0.691    , 0.011    , 0.716    , 0.852    , 0.98     ,
       0.63     , 0.714    , 0.963    , 0.933    , 0.955    , 0.941    ,
       0.596    , 0.632    , 0.972    , 0.987    , 0.164    , 0.878    ,
       0.255    , 0.137    , 0.219    , 0.02     , 0.134    , 0.859    ,
       0.05     , 0.125    , 0.077    , 0.615    , 0.988    , 0.979    ,
       0.959    , 0.982    , 0.717    , 0.302    , 0.774    , 0.024    ,
       0.021    , 0.035    , 0.01     , 0.006    , 

In [43]:
df['PolyPhen'] = df['PolyPhen'].str.replace(r'[^\d.]+', '')

C:\Users\aisin\AppData\Local\Temp\ipykernel_17644\1614955975.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['PolyPhen'] = df['PolyPhen'].str.replace(r'[^\d.]+', '')


In [45]:
df['PolyPhen'] = pd.to_numeric(df['PolyPhen'], errors='coerce')

In [46]:
df['PolyPhen'] = df['PolyPhen'].fillna(df['PolyPhen'].mean())

In [51]:
df['SIFT'].unique()

array([0.14, 0.04, 0.13, 0.1 , 0.05, 0.11, 0.01, 0.  , 1.  , 0.27, 0.36,
       0.09, 0.15, 0.23, 0.78, 0.75, 0.81, 0.73, 0.89, 0.64, 0.63, 0.08,
       0.03, 0.06, 0.07, 0.37, 0.02, 0.18, 0.2 , 0.28, 0.21, 0.24, 0.25,
       0.17, 0.7 , 0.29, 0.35, 0.3 , 0.34])

In [49]:
df['SIFT'] = df['SIFT'].str.replace(r'[^\d.]+', '')

C:\Users\aisin\AppData\Local\Temp\ipykernel_17644\2125992723.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['SIFT'] = df['SIFT'].str.replace(r'[^\d.]+', '')


In [50]:
df['SIFT'] = pd.to_numeric(df['SIFT'], errors='coerce')

In [36]:
# Check how many rows for each class
df['CLIN_SIG'].value_counts()

likely_benign        144
benign               135
likely_pathogenic    106
pathogenic            75
Name: CLIN_SIG, dtype: int64

In [31]:
df['CLIN_SIG'] = df['CLIN_SIG'].replace({'likely_benign&benign/likely_benign':'likely_benign'})

In [35]:
# drop all rows with not_provided in CLIN_SIG
df = df[~(df == 'conflicting_interpretations_of_pathogenicity&uncertain_significance&benign').any(axis=1)]

In [ ]:
# Save the prepared dataset
df.to_csv("Mutagenicity.csv", index=False)

In [ ]:
# Initialize and train the model

In [53]:
# Separate features and target variable
X = df.drop('CLIN_SIG', axis=1)
y = df['CLIN_SIG']

In [54]:
# convert strings in Clin_Sig to int
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [55]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [57]:
y_train

array([3, 0, 2, 0, 1, 2, 0, 1, 1, 3, 2, 1, 2, 3, 0, 3, 0, 2, 2, 1, 1, 3,
       0, 1, 0, 2, 2, 0, 0, 2, 1, 1, 1, 1, 2, 0, 1, 2, 0, 3, 3, 0, 2, 1,
       1, 1, 2, 1, 1, 2, 2, 2, 2, 2, 3, 3, 1, 2, 0, 1, 2, 3, 1, 1, 1, 2,
       0, 2, 1, 2, 2, 3, 2, 1, 3, 1, 3, 2, 3, 0, 0, 2, 0, 0, 1, 0, 2, 0,
       1, 2, 1, 3, 2, 3, 3, 0, 2, 1, 2, 3, 1, 1, 0, 0, 1, 2, 2, 2, 1, 0,
       3, 2, 0, 0, 3, 1, 1, 1, 3, 1, 0, 1, 3, 1, 1, 1, 2, 1, 3, 0, 0, 0,
       2, 0, 3, 2, 3, 0, 1, 3, 0, 0, 2, 0, 2, 2, 2, 1, 2, 0, 1, 0, 0, 0,
       1, 1, 0, 0, 2, 2, 0, 1, 1, 2, 3, 1, 2, 3, 0, 0, 2, 2, 2, 1, 0, 2,
       2, 0, 0, 0, 2, 2, 1, 0, 3, 2, 1, 0, 1, 2, 2, 0, 1, 2, 3, 0, 0, 3,
       2, 3, 0, 2, 0, 0, 1, 1, 0, 2, 1, 2, 1, 1, 3, 3, 1, 2, 0, 0, 1, 3,
       0, 1, 2, 1, 2, 0, 1, 1, 0, 1, 3, 1, 0, 1, 3, 1, 0, 1, 2, 3, 0, 1,
       0, 0, 3, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 2, 1, 2, 0, 1, 1, 0, 1,
       1, 0, 1, 1, 0, 1, 3, 1, 0, 3, 0, 3, 1, 1, 1, 0, 1, 1, 3, 1, 0, 1,
       0, 1, 1, 0, 1, 3, 0, 0, 3, 0, 1, 1, 2, 0, 2,

In [58]:
y_test

array([2, 3, 1, 1, 2, 1, 1, 0, 1, 2, 3, 3, 2, 0, 0, 1, 1, 1, 2, 1, 0, 2,
       0, 0, 1, 0, 2, 2, 0, 1, 0, 1, 3, 0, 3, 0, 1, 1, 2, 0, 1, 0, 1, 2,
       0, 1, 2, 1, 1, 2, 2, 3, 0, 0, 3, 1, 0, 1, 2, 1, 1, 1, 2, 0, 2, 2,
       0, 0, 0, 1, 3, 3, 3, 2, 0, 1, 1, 1, 1, 1, 2, 2, 3, 0, 0, 0, 2, 0,
       0, 3, 1, 1])

In [ ]:
# start with the Gradient Boosting

In [59]:
# Define the XGBoost classifier
xgb_model = xgb.XGBClassifier()

In [60]:
# Train the classifier
xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [61]:
# Make predictions on the test set
y_pred = xgb_model.predict(X_test)

In [62]:
# Inverse transform the numeric labels back to string labels (if needed)
y_pred = label_encoder.inverse_transform(y_pred)
y_test = label_encoder.inverse_transform(y_test)

In [64]:
# Evaluate the accuracy of the model
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 97.83%


In [69]:
xgb_model.save_model("C:/Users/aisin/Desktop/Files/For Python/Mutagenecity dataset/grad_boost_CLINSIG.txt")